# Basic Tensors

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
x = np.arange(0, 25)
x = tf.constant(x)
tf.square(x)

<tf.Tensor: shape=(25,), dtype=int32, numpy=
array([  0,   1,   4,   9,  16,  25,  36,  49,  64,  81, 100, 121, 144,
       169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576])>

In [3]:
x = tf.reshape(x,(5,5))
x

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])>

In [4]:
tmp = tf.constant([1,2,3,4],shape=(2,2))
tmp

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>

In [5]:
#shape does not work with tf.Variable
try:
    # This will produce a ValueError
    tf.Variable([1,2,3,4], shape=(2,2))
except ValueError as v:
    # See what the ValueError says
    print(v)

The initial value's shape ((4,)) is not compatible with the explicitly supplied `shape` argument ((2, 2)).


In [6]:
x = tf.cast(x, tf.float32)
x

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.]], dtype=float32)>

In [7]:
#Broadcasting
y = tf.constant(2, shape=(5,5),dtype=tf.float32)
y

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.]], dtype=float32)>

In [8]:
result = tf.multiply(x, y)
result

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[ 0.,  2.,  4.,  6.,  8.],
       [10., 12., 14., 16., 18.],
       [20., 22., 24., 26., 28.],
       [30., 32., 34., 36., 38.],
       [40., 42., 44., 46., 48.]], dtype=float32)>

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2,input_shape=(3,5))
])

In [21]:
model.variables

[<tf.Variable 'dense_5/kernel:0' shape=(5, 2) dtype=float32, numpy=
 array([[ 0.6362537 ,  0.64162254],
        [ 0.55725324,  0.6576071 ],
        [ 0.3180008 ,  0.24014044],
        [ 0.7295371 ,  0.7509128 ],
        [-0.6592019 ,  0.36261773]], dtype=float32)>,
 <tf.Variable 'dense_5/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

# Gradients

In [9]:
%reset -f
import numpy as np
import tensorflow as tf

In [10]:
x = tf.ones((2,2))
with tf.GradientTape() as t:
    # commenting below line does not give any output
    # using tf.Variable instead of ones/constant - This issue wont be there
    t.watch(x)
    y = tf.reduce_sum(x)
    print(y)
    z = tf.square(y)
    print(z)
    
dz_dx = t.gradient(z,x)
print(dz_dx)

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(16.0, shape=(), dtype=float32)
tf.Tensor(
[[8. 8.]
 [8. 8.]], shape=(2, 2), dtype=float32)


In [11]:
# Gradient tape expires after one use, by default
x = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(x)
    y = x * x
    z = y * y
dz_dx = t.gradient(z, x)
print(dz_dx)

tf.Tensor(108.0, shape=(), dtype=float32)


In [12]:
# If you try to compute dy/dx after the gradient tape has expired:
try:
    dy_dx = t.gradient(y, x)  # 6.0
    print(dy_dx)
except RuntimeError as e:
    print("The error message you get is:")
    print(e)

The error message you get is:
GradientTape.gradient can only be called once on non-persistent tapes.


### Persistent Gradient tape

In [13]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x * x
    z = y * y
dz_dx = t.gradient(z, x)
print(dz_dx)

tf.Tensor(108.0, shape=(), dtype=float32)


In [14]:
# You can still compute dy/dx because of the persistent flag.
dy_dx = t.gradient(y, x)  # 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [15]:
# Drop the reference to the tape
del t

### Higher order derivative - Nested gradient Tape

In [16]:
# watch is not required with tf.Variable
x = tf.Variable(1.0)

with tf.GradientTape() as tape_2:
    #tape_2.watch(x)
    with tf.GradientTape() as tape_1:
        #tape_1.watch(x)
        y = x * x * x
    #first derivative calculation can be within the with block
    #If outside outer with block then it wont persist 2nd gradient calculation
    #It wont work even if you make both gradientTape persistent
    dy_dx = tape_1.gradient(y, x)
#2nd derivative can be indented as much as the first derivative
d2y_dx2 = tape_2.gradient(dy_dx, x)

print(dy_dx)
print(d2y_dx2)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


In [17]:
#If outside outer with block then it wont persist 2nd gradient calculation
#It wont work even if you make both gradientTape persistent
x = tf.Variable(1.0)

with tf.GradientTape(persistent=True) as tape_2:
    with tf.GradientTape(persistent=True) as tape_1:
        y = x * x * x
dy_dx = tape_1.gradient(y, x)
d2y_dx2 = tape_2.gradient(dy_dx, x)

print(dy_dx)
print(d2y_dx2)

tf.Tensor(3.0, shape=(), dtype=float32)
None
